In [51]:
from datasets import load_dataset
import pandas as pd
import os
from dotenv import load_dotenv
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pandas
import openai
from openai import APIError
import os
import json
import re
import numpy as np
from sklearn.cluster import KMeans
from pprint import pprint
from pathlib import Path
import tiktoken
from sentence_transformers import SentenceTransformer


# Load the .env file
load_dotenv()

# Access the variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY
pd.set_option('display.max_colwidth', None)  # None means no limit on column width
pd.set_option('display.width', 300)  

In [52]:
from datasets import Dataset
def get_sts_dataset():
    df = pd.read_parquet("datasets/STS.parquet")
    data_dict = {"sentence1": [], "sentence2": [], "score": []}
    for _, row in df.iterrows():
        data_dict["sentence1"].append(row["sentence1"])
        data_dict["sentence2"].append(row["sentence2"])
        data_dict["score"].append(row["score"] / 5.0)
    # Create a Hugging Face Dataset
    return Dataset.from_dict(data_dict)

In [54]:
import json

# Define the path to the .jsonl file
file_path = 'commands/results_STS.jsonl'

# Open and load the .jsonl file
with open(file_path, 'r') as file:
    data = [json.loads(line) for line in file]

In [55]:
data

[{'id': 'batch_req_66f2ec4117a481909c404fc38e0b95ac',
  'custom_id': 'task-0024-4fedc461-f913-48ef-bd0b-a9c8502f8589',
  'response': {'status_code': 200,
   'request_id': '7609b24c93d879253af932e4d2309806',
   'body': {'id': 'chatcmpl-AB2gMJ6wosebVnSrO9TnVwGgB5bTh',
    'object': 'chat.completion',
    'created': 1727195282,
    'model': 'gpt-3.5-turbo-0125',
    'choices': [{'index': 0,
      'message': {'role': 'assistant',
       'content': '{\n    "sentence": "Devojčica češlja kosu."\n}',
       'refusal': None},
      'logprobs': None,
      'finish_reason': 'stop'}],
    'usage': {'prompt_tokens': 575,
     'completion_tokens': 20,
     'total_tokens': 595,
     'completion_tokens_details': {'reasoning_tokens': 0}},
    'system_fingerprint': None}},
  'error': None},
 {'id': 'batch_req_66f2ec412aa0819081d70d3aef2c6a75',
  'custom_id': 'task-0033-b829a68f-c8cf-4362-8bcb-82ae1939292a',
  'response': {'status_code': 200,
   'request_id': '8cb1ea3b2154c5c5feb886f0be6b6618',
   'body'

In [56]:
ids = []
results = []

# Parse the data
for item in data:
    # Extract the second part of custom_id (after "task-")
    custom_id = item['custom_id'].split('-')[1]
    ids.append(custom_id)
    
    # Extract content from message
    message_content = item['response']['body']['choices'][0]['message']['content']
    results.append(message_content)
    parsed_content = [json.loads(result) for result in results]

# Print or use the results
print("IDs:", ids)
print("Results:", parsed_content)

IDs: ['0024', '0033', '0045', '0063', '0066', '0074', '0076', '0082', '0092', '0095', '0096', '0100', '0103', '0107', '0124', '0127', '0128', '0134', '0137', '0139', '0146', '0150', '0156', '0162', '0169', '0170', '0171', '0174', '0181', '0183', '0197', '0209', '0229', '0232', '0242', '0244', '0248', '0251', '0253', '0258', '0266', '0276', '0279', '0281', '0302', '0308', '0310', '0313', '0321', '0326', '0330', '0333', '0337', '0341', '0343', '0346', '0353', '0356', '0358', '0363', '0370', '0383', '0389', '0396', '0400', '0411', '0412', '0421', '0425', '0435', '0436', '0452', '0459', '0463', '0465', '0470', '0475', '0487', '0491', '0509', '0521', '0533', '0540', '0544', '0545', '0548', '0549', '0554', '0557', '0563', '0572', '0579', '0590', '0592', '0599', '0602', '0606', '0615', '0622', '0628', '0636', '0647', '0648', '0656', '0658', '0662', '0663', '0664', '0665', '0674', '0677', '0679', '0683', '0698', '0702', '0703', '0714', '0720', '0726', '0730', '0733', '0734', '0739', '0741', '0

In [67]:
final = []
for r in parsed_content:
    res = r['sentence']
    final.append(res)
    
data = []
for i in range(len(ids)):
    data.append({
        f'id': ids[i],             # Create dynamic key like 'id1', 'id2'
        f'sentence': final[i]    # Create dynamic key like 'sentence1', 'sentence2'
    })

# Output the result
print(data)

[{'id': '0024', 'sentence': 'Devojčica češlja kosu.'}, {'id': '0033', 'sentence': 'Grupa dečaka igra fudbal na plaži.'}, {'id': '0045', 'sentence': 'Jedna žena meri gležanj druge žene.'}, {'id': '0063', 'sentence': 'Čovek secka krastavac.'}, {'id': '0066', 'sentence': 'Čovek svira klavijaturu.'}, {'id': '0074', 'sentence': 'Žena seče tofu.'}, {'id': '0076', 'sentence': 'Čovek vozi bicikl.'}, {'id': '0082', 'sentence': 'Čovek svira gitaru.'}, {'id': '0092', 'sentence': 'Dama svira gitaru.'}, {'id': '0095', 'sentence': 'Čovek svira trubu.'}, {'id': '0096', 'sentence': 'Čovek svira trubu.'}, {'id': '0100', 'sentence': 'Čovek seče luk.'}, {'id': '0103', 'sentence': 'Jedan čovek priča.'}, {'id': '0107', 'sentence': 'Čovek seče ribu.'}, {'id': '0124', 'sentence': 'Čovek seče kiflu.'}, {'id': '0127', 'sentence': 'Čovek svira klavijaturu.'}, {'id': '0128', 'sentence': 'Panda klizi niz tobogan.'}, {'id': '0134', 'sentence': 'Čovek svira gitaru.'}, {'id': '0137', 'sentence': 'Čovek udara ženu.'}

In [58]:
sts = load_dataset("mteb/stsbenchmark-sts")

In [59]:
sts

DatasetDict({
    train: Dataset({
        features: ['split', 'genre', 'dataset', 'year', 'sid', 'score', 'sentence1', 'sentence2'],
        num_rows: 5749
    })
    validation: Dataset({
        features: ['split', 'genre', 'dataset', 'year', 'sid', 'score', 'sentence1', 'sentence2'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['split', 'genre', 'dataset', 'year', 'sid', 'score', 'sentence1', 'sentence2'],
        num_rows: 1379
    })
})

In [60]:
sts_df_og = pd.DataFrame(sts['test'])
sts_df_og.head(2)

,split,genre,dataset,year,sid,score,sentence1,sentence2
0,test,main-captions,MSRvid,2012test,0024,2.5,A girl is styling her hair.,A girl is brushing her hair.
1,test,main-captions,MSRvid,2012test,0033,3.6,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.


In [73]:
sts_srb = pd.DataFrame({
    'sid': ids,
    'sentence2_srb': final
})

In [79]:
sts_df_og['sentence2_srb'] = final

In [81]:
sts_df_og.rename(columns={"sentence2":"sentence2_eng"}, inplace=True)
sts_df_og.head()

,split,genre,dataset,year,sid,score,sentence1,sentence2_eng,sentence2_srb
0,test,main-captions,MSRvid,2012test,0024,2.5,A girl is styling her hair.,A girl is brushing her hair.,Devojčica češlja kosu.
1,test,main-captions,MSRvid,2012test,0033,3.6,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.,Grupa dečaka igra fudbal na plaži.
2,test,main-captions,MSRvid,2012test,0045,5.0,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.,Jedna žena meri gležanj druge žene.
3,test,main-captions,MSRvid,2012test,0063,4.2,A man is cutting up a cucumber.,A man is slicing a cucumber.,Čovek secka krastavac.
4,test,main-captions,MSRvid,2012test,0066,1.5,A man is playing a harp.,A man is playing a keyboard.,Čovek svira klavijaturu.


In [82]:
sts_df_og.sample(20)

,split,genre,dataset,year,sid,score,sentence1,sentence2_eng,sentence2_srb
852,test,main-forums,answer-answer,2016,1389,5.0,There are three options:,There are only three options:,Postoje samo tri opcije:
51,test,main-captions,MSRvid,2012test,0333,1.4,A woman is slicing some tomatoes.,A woman is chopping a potato.,Žena secka krompir.
169,test,main-captions,MSRvid,2012train,0238,2.6,A girl is coming up on a car.,A girl jumps on top of a car.,Devojčica skače na vrh automobila.
1278,test,main-news,headlines,2015,0210,0.4,China sacks top economic official,China FM hosts senior ASEAN officials,Kineski ministar spoljnih poslova domaćin je visokih zvaničnika ASEAN-a.
702,test,main-forums,answer-answer,2016,0445,4.0,How should I proceed about this?,So how should I do this?,Kako bih trebalo ovo da uradim?
1242,test,main-news,headlines,2014,0473,2.6,Two firefighters killed in New York ambush,Two New York firefighters in guarded condition after deadly ambush,Dvojica vatrogasaca iz Njujorka u stabilnom stanju nakon smrtonosnog zasede.
657,test,main-forums,answer-answer,2016,0212,2.0,You don't have to know.,You have no need to do anything.,Nemate potrebe da bilo šta radite.
607,test,main-captions,track5.en-en,2017,0216,1.8,A little girl in a white blouse and blue pants frolics on the grass.,A woman in a green shirt and white pants golfs on the green.,Žena u zelenoj majici i belim pantalonama igra golf na zelenoj površini.
535,test,main-captions,track5.en-en,2017,0062,1.0,A woman is about to paint.,A boy is about to take a picture.,Dečak će uskoro napraviti fotografiju.
283,test,main-captions,images,2014,0236,4.2,a small bird sitting on a branch in winter.,A small bird perched on an icy branch.,Mala ptica je sela na ledeni grančicu.


In [84]:
pth = "datasets/STS_parallel_en_sr.parquet"
sts_df_og.to_parquet(pth)

In [85]:
pd.read_parquet(pth)

,split,genre,dataset,year,sid,score,sentence1,sentence2_eng,sentence2_srb
0,test,main-captions,MSRvid,2012test,0024,2.5,A girl is styling her hair.,A girl is brushing her hair.,Devojčica češlja kosu.
1,test,main-captions,MSRvid,2012test,0033,3.6,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.,Grupa dečaka igra fudbal na plaži.
2,test,main-captions,MSRvid,2012test,0045,5.0,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.,Jedna žena meri gležanj druge žene.
3,test,main-captions,MSRvid,2012test,0063,4.2,A man is cutting up a cucumber.,A man is slicing a cucumber.,Čovek secka krastavac.
4,test,main-captions,MSRvid,2012test,0066,1.5,A man is playing a harp.,A man is playing a keyboard.,Čovek svira klavijaturu.
...,...,...,...,...,...,...,...,...,...
1374,test,main-news,headlines,2016,1354,0.0,"Philippines, Canada pledge to further boost relations",Philippines saves 100 after ferry sinks,Filipini spasavaju 100 ljudi nakon potonuća trajekta.
1375,test,main-news,headlines,2016,1360,1.0,Israel bars Palestinians from Jerusalem's Old City,"Two-state solution between Palestinians, Israel pie in sky",Rešenje sa dve države između Palestinaca i Izraela je kao kule u vazduhu.
1376,test,main-news,headlines,2016,1368,1.0,How much do you know about Secret Service?,Lawmakers from both sides express outrage at Secret Service,Zakonodavci sa obe strane izražavaju ogorčenje zbog Tajne službe.
1377,test,main-news,headlines,2016,1420,0.0,Obama Struggles to Soothe Saudi Fears As Iran Talks Resume,Myanmar Struggles to Finalize Voter Lists for Sunday Polls,Mijanmar se bori da završi popis birača za nedeljne izbore.
